In [1]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
import copy



from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP, ADAM, L_BFGS_B, GSLS, NELDER_MEAD, NFT, P_BFGS
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit import Delay
from qiskit import IBMQ, Aer, transpile, schedule as build_schedule


from qiskit.providers.aer import AerSimulator
home_stabilizer= AerSimulator(method='stabilizer',max_parallel_experiments=0,max_parallel_shots=1)
home_sim2 = AerSimulator()

q_svh = QuantumInstance(backend=home_stabilizer, 
                                   shots=1, 
                      )





q_ideal = QuantumInstance(backend=home_sim2, 
                                   shots=8192, 
                      )

def modify_gates2(circuit):
    qc = QuantumCircuit(circuit.num_qubits)
    A = ParameterVector('A',1)
    first=0
    for item in circuit:
        if(first==0):
            first=1
            qc.delay(A[0],item[1])
        qc.append(item[0],item[1],item[2])

    #qc.measure_active()
    return qc

def modify_gates(circuit):
    qc = QuantumCircuit(circuit.num_qubits)
    A = ParameterVector('A',1)
    first=0
    for item in circuit:
        #print(item[0].qasm())
        if(item[0].qasm() == "rz(-pi)" or item[0].qasm() == "rz(pi)"):
            qc.z(item[1])
            if(first==0):
                first=1
                qc.delay(A[0],item[1])
        elif(item[0].qasm() == "rz(-pi/2)" or item[0].qasm() == "rz(3*pi/2)"):
            qc.s(item[1])
            qc.z(item[1])
            if(first==0):
                first=1
                qc.delay(A[0],item[1])
        elif(item[0].qasm() == "rz(-3*pi/2)" or item[0].qasm() == "rz(pi/2)"):
            qc.s(item[1])
            if(first==0):
                first=1
                qc.delay(A[0],item[1])
        elif(item[0].qasm()!="measure" and item[0].qasm()!="barrier" ):
            qc.append(item[0],item[1],item[2])

    #qc.measure_active()
    return qc


def get_qubit_op(dist): 
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 "+ str(dist) , unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = repulsion_energy #+ energy_shift
    return qubitOp, num_particles, num_spin_orbitals, shift #- can add shift latre if required - doesnt really matter


length=0.74
distances = [1.0*length]
exact_energies = []
vqe_energies = []
shifts = []
init = []
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist) #shift
    result = NumPyEigensolver(qubitOp).run()
    shifts.append(shift)
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = EfficientSU2(qubitOp.num_qubits, entanglement="linear",reps=1)
    #print(var_form.num_parameters)
    qc = initial_state.construct_circuit() #+ var_form
    qc = modify_gates2(qc)
    vqe = VQE(qubitOp, qc, optimizer=COBYLA(maxiter=1), initial_point = [0],include_custom='True')
    ret = vqe.run(q_ideal)
    vqe_result = np.real(ret['eigenvalue'])
    print(dist,np.real(result.eigenvalues[0]) + shift, vqe_result +shift,shift)


/opt/anaconda3/envs/env_6_2021/lib/python3.9/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/opt/anaconda3/envs/env_6_2021/lib/python3.9/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')
/opt/anaconda3/envs/env_6_2021/lib/python3.9/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https

0.74 -1.1372838344885015 -1.116759307396425 0.7151043390810812


In [2]:
def run_func(X):
    x0 = X['x0']    
    x1 = X['x1']
    x2 = X['x2']
    x3 = X['x3']    
    x4 = X['x4']
    x5 = X['x5']    
    x6 = X['x6']    
    x7 = X['x7']


 
    xarray=[x0,x1,x2,x3,x4,x5,x6,x7
           ]
    nparams=8 
    params=[0]*nparams
    for i in range(8): 
        params[i] = xarray[i]*np.pi/2

    checker=0
    for item in xarray:
        if(item != 0):
            checker=1
    if(checker == 0):
        return 0    
        
    var_form = EfficientSU2(qubitOp.num_qubits, entanglement="linear",reps=1)
    vad0 = var_form.assign_parameters(params)
    tcirc = transpile(vad0,basis_gates=['id', 'rz', 'sx', 'x', 'cx', 'reset'])
    tcirc = modify_gates(tcirc)

    
    try:
        vqe = VQE(qubitOp, tcirc, optimizer=COBYLA(maxiter=1), initial_point = [0],include_custom='True',quantum_instance=q_svh)
        ret = vqe.run()
        cost = np.real(ret['eigenvalue']) + shift
    except:
        return 0

    return cost

In [6]:
import json


scenario = {}
scenario["application_name"] = "H2-shots-BO2-long-1x-trial"
scenario["optimization_objectives"] = ["value"]


number_of_RS = 100
scenario["design_of_experiment"] = {}
scenario["design_of_experiment"]["number_of_samples"] = number_of_RS
scenario["optimization_iterations"] = 5000

scenario["models"] = {}
scenario["models"]["model"] = "random_forest"


scenario["input_parameters"] = {}

x0 = {}
x0["parameter_type"] =  "ordinal"
x0["values"] = [0, 1,2,3]
x0["parameter_default"] = 0

x1 = {}
x1["parameter_type"] =  "ordinal"
x1["values"] = [0, 1,2,3]
x1["parameter_default"] = 0


x2 = {}
x2["parameter_type"] =  "ordinal"
x2["values"] = [0, 1,2,3]
x2["parameter_default"] = 0

x3 = {}
x3["parameter_type"] =  "ordinal"
x3["values"] = [0, 1,2,3]
x3["parameter_default"] = 0

x4 = {}
x4["parameter_type"] =  "ordinal"
x4["values"] = [0, 1,2,3]
x4["parameter_default"] = 0

x5 = {}
x5["parameter_type"] =  "ordinal"
x5["values"] = [0, 1,2,3]
x5["parameter_default"] = 0

x6 = {}
x6["parameter_type"] =  "ordinal"
x6["values"] = [0, 1,2,3]
x6["parameter_default"] = 0

x7 = {}
x7["parameter_type"] =  "ordinal"
x7["values"] = [0, 1,2,3]
x7["parameter_default"] = 0




scenario["input_parameters"]["x0"] = x0
scenario["input_parameters"]["x1"] = x1
scenario["input_parameters"]["x2"] = x2
scenario["input_parameters"]["x3"] = x3
scenario["input_parameters"]["x4"] = x4
scenario["input_parameters"]["x5"] = x5
scenario["input_parameters"]["x6"] = x6
scenario["input_parameters"]["x7"] = x7


with open("H2-shots-BO2-long-1x-trial.json", "w") as scenario_file:
    json.dump(scenario, scenario_file, indent=4)

In [7]:
f = open("H2-shots-BO2-long-1x-trial.json", "r")
text = f.read()
print(text, flush=True)
f.close()

{
    "application_name": "H2-shots-BO2-long-1x-trial",
    "optimization_objectives": [
        "value"
    ],
    "design_of_experiment": {
        "number_of_samples": 100
    },
    "optimization_iterations": 5000,
    "models": {
        "model": "random_forest"
    },
    "input_parameters": {
        "x0": {
            "parameter_type": "ordinal",
            "values": [
                0,
                1,
                2,
                3
            ],
            "parameter_default": 0
        },
        "x1": {
            "parameter_type": "ordinal",
            "values": [
                0,
                1,
                2,
                3
            ],
            "parameter_default": 0
        },
        "x2": {
            "parameter_type": "ordinal",
            "values": [
                0,
                1,
                2,
                3
            ],
            "parameter_default": 0
        },
        "x3": {
            "parameter_type": "o

In [ ]:
import sys
import os
from hypermapper import optimizer
stdout = sys.stdout # Jupyter uses a special stdout and HyperMapper logging overwrites it. Save stdout to restore later

# Call HyperMapper 
optimizer.optimize("H2-shots-BO2-long-1x-trial.json", run_func)
sys.stdout = stdout

Design of experiment phase, number of new doe samples = 100 .......
x0,x1,x2,x3,x4,x5,x6,x7,value,Timestamp
0,0,0,0,0,0,0,0,0,16
2,2,1,1,2,0,2,1,-0.3495628949862579,47
3,0,1,0,1,2,0,0,-0.33831673783543703,74
1,2,1,0,0,1,0,0,-0.33831673783543703,97
2,2,3,1,0,3,1,1,-0.7331611011913415,117
0,0,2,0,3,1,3,2,-0.33831673783543703,137
2,0,2,3,2,0,3,0,0.4626181460271928,155
2,2,2,1,0,3,3,3,-0.7331611011913415,176
0,1,0,2,1,2,3,2,-0.33831673783543703,196
0,3,0,2,1,0,3,0,-0.33831673783543703,215
1,1,2,2,1,1,0,0,-1.116759307396425,237
2,0,0,0,0,2,1,2,-1.116759307396425,262
2,0,3,3,1,3,2,0,-0.15710627582023984,285
3,1,0,3,0,3,3,3,-0.33831673783543703,307
0,2,1,2,1,1,1,3,-0.33831673783543703,330
0,2,0,1,0,2,1,1,-0.3495628949862579,349
1,0,3,3,1,2,1,3,-0.33831673783543703,369
3,3,0,0,1,1,3,0,-1.116759307396425,389
3,1,2,2,3,3,3,3,-0.3495628949862579,408
2,0,1,3,0,2,1,0,-1.116759307396425,427
1,2,0,1,1,2,1,0,-0.33831673783543703,445
2,3,3,1,3,0,0,1,-0.5195271998506343,464
2,1,1,1,3,2,1,0,-0.3383167378